# AIMO Prize 3 - OOP Baseline Submission (v2)

This notebook demonstrates how to use the Object-Oriented Baseline with **Few-Shot Prompting** and **Majority Voting**.

## Setup Instructions
1. **Add Utility Script**: Upload `src/kaggle_baseline.py` as a Dataset (e.g., named `aimo-pp3-modules`).
2. **Add Model**: Search for and add the model `Qwen/Qwen2.5-Math-7B-Instruct` (or a quantized version like `Qwen2.5-Math-7B-Instruct-GGUF`).
   - Expected Path: `/kaggle/input/qwen2.5-math-7b-instruct` (Adjust in config if different).
3. **Run**: Execute the cells below.

In [ ]:
import sys
import os

# 1. Import the OOP Baseline Module
sys.path.append('/kaggle/input/aimo-pp3-modules')
sys.path.append(os.path.join(os.getcwd(), 'src')) # Fallback for local dev

try:
    from kaggle_baseline import CompetitionConfig, AIMSolver, MockLLM, LocalVLLM
    print("✅ Successfully imported OOP Baseline modules.")
except ImportError as e:
    print(f"❌ Import Failed: {e}. Make sure the 'aimo-pp3-modules' dataset is attached.")

In [ ]:
# 2. Configure Environment & Model
config = CompetitionConfig()
config.n_repetitions = 1 # Set to 3-5 for Majority Voting (requires GPU)

print(f"Environment: {'Kaggle' if config.is_kaggle else 'Local'}")
print(f"Model Path: {config.model_path}")

# Initialize Model
if config.is_kaggle and os.path.exists(config.model_path):
    # Uncomment for Real Model (vLLM)
    # llm = LocalVLLM(config.model_path)
    print("⚠️ Using MockLLM. Uncomment LocalVLLM lines to use the real model.")
    llm = MockLLM()
else:
    print("⚠️ Model not found or running locally. Using MockLLM.")
    llm = MockLLM()

# Initialize Solver
solver = AIMSolver(config, llm)

## Local Evaluation (Dry Run)

In [ ]:
import pandas as pd

csv_path = config.get_dataset_path("reference.csv")
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path).head(3)
    print(f"Testing on {len(df)} samples...")
    for idx, row in df.iterrows():
        ans = solver.solve(row['problem'])
        print(f"Problem {row['id']}: Pred={ans}, True={row['answer']}")
else:
    print("No reference data found for quick test.")

## Submission Loop (API)

In [ ]:
import aimo_3_gateway as aimo

try:
    env = aimo.make_env()
    iter_test = env.iter_test()
    print("Connected to AIMO Gateway.")
    
    for test, sample_submission in iter_test:
        for index, row in test.iterrows():
            result = solver.solve(row['problem'])
            sample_submission.loc[index, 'answer'] = result
            
        env.predict(sample_submission)
        print(f"Submitted batch.")
        
except ImportError:
    print("aimo_3_gateway not found. Skipping submission loop (Local Mode).")
except Exception as e:
    print(f"Submission Error: {e}")